### Imports

In [1]:
import requests
import os
from dotenv import load_dotenv
import json
from utils import *

### Variables const , globals , sensible data

In [2]:
load_dotenv()

# constants / sensible data

STOCK_NAME = os.getenv("STOCK_NAME")
COMPANY_NAME = os.getenv("COMPANY_NAME")
STOCK_ENDPOINT = os.getenv("STOCK_ENDPOINT")
NEWS_ENDPOINT = os.getenv("NEWS_ENDPOINT")
STOCK_API_KEY = os.getenv("STOCK_API_KEY")
NEWS_API_KEY = os.getenv("NEWS_API_KEY")

In [3]:
# globals
stockFileName = "JSON/intraday.json"
newsFileName = "JSON/everything.json"
#
# stockList = loadJSONFile(stockFileName)
# newsList = loadJSONFile(newsFileName)
#
listStocks = []
listNews = []
#
paramsStocks = {
    'function' : 'TIME_SERIES_INTRADAY',
    'symbol' : STOCK_NAME,
    'interval' : '60min',
    'apikey' : STOCK_API_KEY
}
 
#
paramsNews = '''

'''

### Functions

In [6]:
def exportToJSONIntraDay():
    response = requests.get(url=STOCK_ENDPOINT , params= paramsStocks)
    if response.status_code == 200:
        stocks = response.json()['Time Series (60min)']
        for item in stocks.items():
            listStocks.append(item)

In [21]:
def getIntraDay():
    params = {
        'function' : 'TIME_SERIES_INTRADAY',
        'symbol' : STOCK_NAME,
        'interval' : '60min',
        'apikey' : STOCK_API_KEY,
        'outputsize' : 'compact'
    }
    listIntraDay = []
    response = requests.get(url=STOCK_ENDPOINT, params=params)
    if response.status_code == 200:
        data = response.json()['Time Series (60min)']
        for item in data.items():
            listIntraDay.append({
                'datetime' : item[0],
                'open' : item[1]['1. open'],
                'high' : item[1]['2. high'],
                'low' : item[1]['3. low'],
                'close' : item[1]['4. close'],
                'volume' : item[1]['5. volume']
            })
    return listIntraDay


In [25]:
def findDataByDate(date ,intraday):
    find = [item for item in intraday if item['datetime'] == date]
    if find:
        return find

In [32]:

def getDateDict(datetime, intraday):
    for day in intraday:
        if day["datetime"] == datetime:
            return day
    return None

In [45]:

def getStatisticData(dateDict):
    variation = float(dateDict['close'])-float(dateDict['open'])
    variation_perc = (variation / float(dateDict['close']))*100
    return variation, variation_perc

In [46]:

def getDateStock(datetime, intraday):
    dateDict = getDateDict(datetime, intraday)
    variation, variation_perc = getStatisticData(dateDict)
    print(f"""
          Date: {datetime}
          Stock: {COMPANY_NAME}
          Abertura: {dateDict['open']}
          Fecho: {dateDict['close']}
          Volume: {dateDict['volume']}
          Variação: {variation :.2f}$ ({variation_perc :.2f}%)
          """)

In [43]:

def getWeekDict(intraWeek):
    open = 0
    close = 0
    for day in intraWeek:
        open = day['open']
        close = day['close']
        if day["open"] > open:
            open = day['open']
        if day["close"] > close:
            close = day['close']
        return {"open":open , "close": close}
    return None

In [44]:
def printWeekStock(intraWeek):
    weekDict = getWeekDict(intraWeek)
    variation, variation_perc = getStatisticData(weekDict)
    print(f"""
          Stock: {COMPANY_NAME}
          Abertura: {weekDict['open']}
          Fecho: {weekDict['close']}
          Variação: {variation :.2f}$ ({variation_perc :.2f}%)
          """)

In [1]:
def getIntraweek():
    params = {
        'function' : 'TIME_SERIES_WEEKLY',
        'symbol' : STOCK_NAME,
        'apikey' : STOCK_API_KEY,
    }
    listIntraWeek =[]
    response = requests.get(url=STOCK_ENDPOINT, params=params)
    if response.status_code == 200:
        data = response.json()['Time Series (60min)']
        for item in data.items():
            listIntraWeek.append({
                'datetime' : item[0],
                'open' : item[1]['1. open'],
                'close' : item[1]['4. close']
            })
    return listIntraWeek

### Call functions

In [2]:
intraWeek = getIntraweek()
saveJSONFile("JSON/intraWeek.json", intraWeek)

NameError: name 'STOCK_NAME' is not defined

In [3]:
intraday = getIntraDay()
saveJSONFile("JSON/intraday.json", intraday)

NameError: name 'getIntraDay' is not defined

In [31]:
dateToFind = "2024-05-24 19:00:00"
print(findDataByDate(dateToFind ,intraday))

[{'datetime': '2024-05-24 19:00:00', 'open': '178.9500', 'high': '179.0000', 'low': '178.7600', 'close': '178.8500', 'volume': '69420'}]


In [37]:
getDateStock("2024-05-24 19:00:00", intraday)


          Date: 2024-05-24 19:00:00
          Stock: Tesla Inc
          Abertura: 178.9500
          Fecho: 178.8500
          Volume: 69420
          Variação: -0.10$ (-0.06%)
          
